<a href="https://colab.research.google.com/github/dntjr41/Community_Detection_Paper/blob/master/Check_Neural.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>